<a href="https://colab.research.google.com/github/coder160/cuadernos/blob/main/Vision_Artificial/HuggingFace/CLASIFICADOR__VIDEOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Video Classification

La clasificación de videos consiste en etiquetar secuencias de video en categorías específicas. Esto es crucial en aplicaciones como la detección de actividad humana en videos de vigilancia y la organización de contenido multimedia en redes sociales.



Documentación:

* **Transformers** - https://huggingface.co/docs/transformers/v4.32.1/en/index

* **ViViT** - Video Vision Transformers - https://huggingface.co/docs/transformers/main/model_doc/vivit#video-vision-transformer-vivit

* **Google/ViViT-b-16x2** - Modelo - https://huggingface.co/google/vivit-b-16x2-kinetics400

* **Papper Original** - Anurag Arnab - https://arxiv.org/abs/2103.15691

* **ViT** - Vision Transformers - https://huggingface.co/docs/transformers/v4.27.0/model_doc/vit

* **NumPy** - Análisis numérico - https://pypi.org/project/numpy/

* **Av** - Video Encoder - https://pypi.org/project/av/

* **Pexels** - Videos Stack - https://www.pexels.com/search/videos/stack/


### **Ejemplo**:

>Para nuestro ejemplo, seguiremos la guía proporcionada por **HuggingFace** para la clasificación de videos **<a href="https://huggingface.co/docs/transformers/main/model_doc/vivit#video-vision-transformer-vivit"> ViViT </a>**- Video Vision Transformer.
>
>Utilizaremos el modelo **<a href="https://huggingface.co/google/vivit-b-16x2-kinetics400"> Google/ViViT-b-16x2Kinetics400</a>** basado en el <a href="https://arxiv.org/abs/2103.15691"> papper</a> de **Anurag Arnab** y en el modelo <a href="https://huggingface.co/docs/transformers/v4.27.0/model_doc/vit"> ViT</a> de **Transformers**.


*Mas información en:*

https://huggingface.co/google/vivit-b-16x2-kinetics400

In [35]:
#@title **Instalador Principal**

#@markdown # **Importante**
#@markdown **Recuerda instalar primero la librería principal junto con todos sus componentes.**

from IPython.display import clear_output

#Instalar el módulo de entornos virtuales venv de python
!apt install python3.10-venv

#Crear nuestro entorno virtual
!python -m venv .env

#Activamos nuestro entorno virtual
!source .env/bin/activate

#Instalamos todas las dependencias de Transformers necesarias
!pip install transformers
!pip install 'transformers[torch]'
!pip install 'transformers[tf-cpu]'
!pip install 'transformers[flax]'
!pip install av
!pip install numpy
clear_output()

#Realizamos una prueba para verificar que el código funciona
print("Instalación correcta")

Instalación correcta


In [36]:
#@title **Ejemplo práctico**

#configuramos numpy para resultados consistentes
np.random.seed(0)

#Función auxiliar para decodificar el video con av
def decodificar_video(video_container, indices):
  #creamos un objeto donde guardaremos los frames decodificados
  frames = []
  #posicionamos el video en el primer frame '0'
  video_container.seek(0)
  #indicamos los límites del video
  primer_indice = indices[0]
  ultimo_indice = indices[-1]
  #decodificamos el video y lo guardamos en frames, en formato de colores rgb
  for indice, frame in enumerate(video_container.decode(video=0)):
    if indice > ultimo_indice:
        break
    if indice >= primer_indice and indice in indices:
        frames.append(frame)
  #devolvemos un array de cada frame, con formato rgb24
  return np.stack([N.to_ndarray(format="rgb24") for N in frames])

#generamos un indice de nuestro video a partir de los frames recibidos
def sampleo_indices(num_frames, tasa_sampleo, frames_video):
  #indicamos el total de frames a partir de la tasa de sampleo
  total_frames = int(num_frames * tasa_sampleo)
  #asignamos valores aleatorios al último id, y al primer id (menor que el último)
  ultimo_id = np.random.randint(total_frames, frames_video)
  primer_id = ultimo_id - total_frames
  #creamos un arreglo de índices a partir de los ids generados
  indices = np.linspace(primer_id, ultimo_id, num=num_frames)
  #ajustamos y devolvemos los índices, con formato de int64
  indices = np.clip(indices, primer_id, ultimo_id - 1).astype(np.int64)
  return indices

try:
  #@markdown 1. Importamos todas las librerías necesarias
  #@markdown para el procesamiento y clasificacion de Videos
  #@markdown junto con algunas librerías de apoyo
  from transformers import VivitImageProcessor, VivitForVideoClassification
  from huggingface_hub import hf_hub_download
  import numpy as np
  import torch
  import av

  #@markdown 2. Definimos la ruta de nuestra video <br>
  #@markdown *Puedes descargar videos de muestra desde:
  #@markdown https://www.pexels.com/search/videos/stack/*
  ruta_local_video = "/content/pexels-tima-miroshnichenko-6170324 (2160p).mp4" # @param {type:"string"}


  #@markdown 3. Abrimos y decodificamos el video
  # generamos un indices a 24 frames, por 8 secuencias
  video_original = av.open(ruta_local_video)
  indices = sampleo_indices(num_frames = 32,
                            tasa_sampleo= 4,
                            frames_video=video_original.streams.video[0].frames)

  video_array = decodificar_video(indices = indices,
                                  video_container = video_original)
  print("Video Decodificado")

  #@markdown 4. Configuramos nuestro procesador y modelo clasificador
  nombre_procesador = "google/vivit-b-16x2-kinetics400" # @param {type:"string"}
  nombre_modelo = "google/vivit-b-16x2-kinetics400" # @param {type:"string"}
  procesador = VivitImageProcessor.from_pretrained(nombre_procesador)
  modelo = VivitForVideoClassification.from_pretrained(nombre_modelo)
  print(f"Modelo:\t{nombre_modelo}\nProcesador:\t{nombre_procesador}")

  #@markdown 5. Procesamos nuestro video en array
  inputs = procesador(list(video_array), return_tensors="pt")
  with torch.no_grad():
    outputs = modelo(**inputs)
    logits = outputs.logits

  # imprimimos la predicción de acuerdo a las etiquetas precargadas
  prediccion = logits.argmax(-1).item()

  print(modelo.config.id2label[prediccion])


except Exception as error:
  print(f"Error con el video\n{error}")

#@markdown El modelo realiza una predicción

Video Decodificado
Modelo:	google/vivit-b-16x2-kinetics400
Procesador:	google/vivit-b-16x2-kinetics400
LABEL_127
